Verify if we have GPU

In [9]:
print("CUDA available?", torch.cuda.is_available())

if torch.cuda.is_available():
    n_gpus = torch.cuda.device_count()
    print(f"Number of CUDA devices: {n_gpus}")


CUDA available? False


Import necessary things

In [14]:
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import torch, re, json
from sklearn.model_selection import train_test_split
from tqdm import tqdm


Load dataset and split it

In [17]:
print("Loading GSM8K…")
gsm8k = load_dataset("gsm8k", "main")
all_data = gsm8k["train"].shuffle(seed=42).select(range(7000)) 
train_val, test_data = train_test_split(all_data, test_size=200, random_state=42)
train_data, val_data = train_test_split(train_val, train_size=6000, test_size=800, random_state=42)
print(f"Splits → Train: {len(train_data)}, Val: {len(val_data)}, Test: {len(test_data)}")


Loading GSM8K…


TypeError: '<' not supported between instances of 'int' and 'ellipsis'